In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from BinanceCollector import BinanceCollector
import talib

In [4]:
collector = BinanceCollector("ETHUSDT","2021-06-01","2022-12-19","15m")
collector.start_collect()
collector.df.columns = ["Open Time","Open","High","Low","Close","Volume","Close Time",
             "Quote asset volumne","Total Trades","Buyer Base Volume",
             "Taker buy quote asset volume","Ignore"]
data = collector.df

In [5]:
data["Return"] = (data["Close"] - data["Open"])*100/data["Open"]
data["Open Time"] = pd.to_datetime(data["Open Time"],unit="ms")
# Calculate the RSI
data["RSI"] = talib.RSI(data["Close"], timeperiod=14)

# Calculate MA
data["SMA_50"] = data["Close"].rolling(window=50).mean()
data["SMA_200"] = data["Close"].rolling(window=200).mean()

# Calculate the MACD
data["MACD"], data["MACD_Signal"], data["MACD_Hist"] = talib.MACD(data["Close"], fastperiod=12, slowperiod=26, signalperiod=9)

# Initialize the signals
data["Signal_RSI"] = np.where((data["RSI"] > 70) | (data["RSI"] < 30), 1, 0)
data["Signal_MA"] = np.where((data["SMA_50"] > data["SMA_200"]) & (data["SMA_50"].rolling(5).mean() <= data["SMA_200"].rolling(5).mean()), 1, 0)
data["Signal_MACD"] = np.where( (data["MACD"] > data["MACD_Signal"]) & (data["MACD"].shift(1) <= data["MACD_Signal"] ),1,0 )

# Shift the signals one day forward
data["Signal_RSI"] = data["Signal_RSI"].shift(1)
data["Signal_MA"] = data["Signal_RSI"].shift(1)
data["Signal_MACD"] = data["Signal_MACD"].shift(1)

In [6]:
day = 0
leverage = 2
liquidation_factor = 100/leverage
returns = 0
for tmp in np.where( data["Signal_MA"] == 1 )[0]:
    target_price = data["Close"][tmp] * (1 + (1/leverage)/100)
    liquidation_price = data["Close"][tmp] * (1 - (liquidation_factor)/100)
    cur = tmp
    while cur < len(data):
        if data["Close"][cur] >= target_price:
            returns += 1
            break
        elif data["Close"][cur] <= liquidation_price:
#             print(f"Target: {target_price}\nLiquidation: {liquidation_price}")
            returns -= 100
            break
        cur += 1
        
print(f"Returns: {returns}")

Returns: -11578
